<h1><font color="purple">Parte 1:</font> Procurar colunas não relevantes</h1>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import json

with open('dicionario.json', 'r') as f:
    d = json.load(f)
    dicionario = {
        **d['participante'], **d['escola'], **d['especializado'], **d['especifico'],
        **d['recurso'], **d['local_prova'], **d['prova'], **d['redacao'], **d['socioeconomico']
    }

with open('categorias.json', 'r') as f:
    c = json.load(f)
    categoria = {
        **c['participante'], **c['escola'], **c['especializado'], **c['especifico'],
        **c['recurso'], **c['local_prova'], **c['prova'], **c['redacao'], **c['socioeconomico']
    }

In [3]:
nrows=5000
skiprows=0
df = pd.read_csv('dados/MICRODADOS_ENEM_2019.csv', sep=';', nrows=nrows, skiprows=skiprows, encoding="ISO-8859-1")

In [4]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001004627,2019,1506807,Santarém,15,PA,21,M,1,3,...,A,A,A,C,B,A,D,A,B,A
1,190001004628,2019,1504059,Mãe do Rio,15,PA,16,F,1,3,...,A,A,A,B,B,A,B,A,A,A
2,190001004629,2019,1505502,Paragominas,15,PA,18,F,1,1,...,B,A,A,D,B,B,D,A,C,B
3,190001004630,2019,1507706,São Sebastião da Boa Vista,15,PA,23,M,0,3,...,A,A,A,C,A,A,D,A,A,A
4,190001004631,2019,1503903,Juruti,15,PA,23,M,1,3,...,A,A,A,B,A,A,D,A,A,A


<h2>1 - Variáveis que indicam códigos</h2>

In [5]:
codigos = list(df.columns.where(df.columns.str.startswith('CO')).dropna())
print(f'{len(codigos)} colunas')

codigos

13 colunas


['CO_MUNICIPIO_RESIDENCIA',
 'CO_UF_RESIDENCIA',
 'CO_MUNICIPIO_NASCIMENTO',
 'CO_UF_NASCIMENTO',
 'CO_ESCOLA',
 'CO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'CO_MUNICIPIO_PROVA',
 'CO_UF_PROVA',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT']

<center><font color="red"><b>As colunas que indicam os códigos podem ser removidos</b></font></center>

| Variável | Descrição |
|---|---|
| CO_MUNICIPIO_RESIDENCIA | Código do município de residência  |
| CO_UF_RESIDENCIA | Código da Unidade da Federação de residência |
| CO_MUNICIPIO_NASCIMENTO | Código do município de nascimento |
| CO_UF_NASCIMENTO | Código da Unidade da Federação de nascimento |
| CO_ESCOLA | Código da Escola4 |
| CO_MUNICIPIO_ESC | Código do município da escola  |
| CO_UF_ESC | Código da Unidade da Federação da escola |
| CO_MUNICIPIO_PROVA | Código do município da aplicação da prova |
| CO_UF_PROVA | Código da Unidade da Federação da aplicação da prova |
| CO_PROVA_CN | Código do tipo de prova de Ciências da Natureza |
| CO_PROVA_CH | Código do tipo de prova de Ciências Humanas |
| CO_PROVA_LC | Código do tipo de prova de Linguagens e Códigos |
| CO_PROVA_MT | Código do tipo de prova de Matemática |

---

In [6]:
mantem = list(df.columns.where(~df.columns.str.startswith('CO')).dropna())
df[mantem].head()

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001004627,2019,Santarém,PA,21,M,1,3,4,NaN,...,A,A,A,C,B,A,D,A,B,A
1,190001004628,2019,Mãe do Rio,PA,16,F,1,3,1,Paragominas,...,A,A,A,B,B,A,B,A,A,A
2,190001004629,2019,Paragominas,PA,18,F,1,1,1,Paragominas,...,B,A,A,D,B,B,D,A,C,B
3,190001004630,2019,São Sebastião da Boa Vista,PA,23,M,0,3,1,Belém,...,A,A,A,C,A,A,D,A,A,A
4,190001004631,2019,Juruti,PA,23,M,1,3,1,Juruti,...,A,A,A,B,A,A,D,A,A,A


<h2> 2 - Variáveis que indicam gabarito e resposta </h2>

In [7]:
texto = list(df.columns.where(df.columns.str.startswith('TX')).dropna())
print(f'{len(texto)} colunas')

texto

8 colunas


['TX_RESPOSTAS_CN',
 'TX_RESPOSTAS_CH',
 'TX_RESPOSTAS_LC',
 'TX_RESPOSTAS_MT',
 'TX_GABARITO_CN',
 'TX_GABARITO_CH',
 'TX_GABARITO_LC',
 'TX_GABARITO_MT']

<center><font color="red"><b>As colunas que indicam as respostas e gabarito podem ser removidos</b></font></center>

<center>As notas são consequencias das respostas, então não teria motivo usar outros dados se já temos as respostas do candidato.</center>

| Variável | Descrição |
|---|---|
| TX_RESPOSTAS_CN | Vetor com as respostas da parte objetiva da prova de Ciências da Natureza8  |
| TX_RESPOSTAS_CH | Vetor com as respostas da parte objetiva da prova de Ciências Humanas8 |
| TX_RESPOSTAS_LC | Vetor com as respostas da parte objetiva da prova de Linguagens e Códigos9  |
| TX_RESPOSTAS_MT | Vetor com as respostas da parte objetiva da prova de Matemática8  |
| TX_GABARITO_CN | Vetor com o gabarito da parte objetiva da prova de Ciências da Natureza10  |
| TX_GABARITO_CH | Vetor com o gabarito da parte objetiva da prova de Ciências Humanas10 |
| TX_GABARITO_LC | Vetor com o gabarito da parte objetiva da prova de Linguagens e Códigos11  |
| TX_GABARITO_MT | Vetor com o gabarito da parte objetiva da prova de Matemática10 |


In [8]:
mantem = list(df.columns.where(
    ~df.columns.str.startswith('CO') & ~df.columns.str.startswith('TX')
).dropna())

df[mantem].head()

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001004627,2019,Santarém,PA,21,M,1,3,4,NaN,...,A,A,A,C,B,A,D,A,B,A
1,190001004628,2019,Mãe do Rio,PA,16,F,1,3,1,Paragominas,...,A,A,A,B,B,A,B,A,A,A
2,190001004629,2019,Paragominas,PA,18,F,1,1,1,Paragominas,...,B,A,A,D,B,B,D,A,C,B
3,190001004630,2019,São Sebastião da Boa Vista,PA,23,M,0,3,1,Belém,...,A,A,A,C,A,A,D,A,A,A
4,190001004631,2019,Juruti,PA,23,M,1,3,1,Juruti,...,A,A,A,B,A,A,D,A,A,A


<h2> 3 - Variáveis que são questões socioeconômicas </h2>

In [9]:
socio = list(df.columns.where(df.columns.str.match('Q01[3-9]|Q02[01]')).dropna())
print(f'{len(socio)} colunas')

socio

9 colunas


['Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021']

<center><font color="red"><b>As seguintes perguntas não parecem ser tão relevantes</b></font></center>

| Variável | Descrição |
|---|---|
| Q013 | Na sua residência tem freezer (independente ou segunda porta da geladeira)? |
| Q014 | Na sua residência tem máquina de lavar roupa? (o tanquinho NÃO deve ser considerado) |
| Q015 | Na sua residência tem máquina de secar roupa (independente ou em conjunto com a máquina de lavar roupa)? |
| Q016 | Na sua residência tem forno micro-ondas? |
| Q017 | Na sua residência tem máquina de lavar louça? |
| Q018 | Na sua residência tem aspirador de pó? |
| Q019 | Na sua residência tem televisão em cores? |
| Q020 | Na sua residência tem aparelho de DVD? |
| Q021 | Na sua residência tem TV por assinatura? |

In [10]:
mantem = list(df.columns.where(
    ~df.columns.str.startswith('CO') & ~df.columns.str.startswith('TX') &
    ~df.columns.str.match('Q01[3-9]|Q02[01]')
).dropna())

df[mantem].head()

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,...,Q007,Q008,Q009,Q010,Q011,Q012,Q022,Q023,Q024,Q025
0,190001004627,2019,Santarém,PA,21,M,1,3,4,NaN,...,A,B,C,A,A,B,D,A,B,A
1,190001004628,2019,Mãe do Rio,PA,16,F,1,3,1,Paragominas,...,A,B,C,A,A,B,B,A,A,A
2,190001004629,2019,Paragominas,PA,18,F,1,1,1,Paragominas,...,D,E,E,C,A,B,D,A,C,B
3,190001004630,2019,São Sebastião da Boa Vista,PA,23,M,0,3,1,Belém,...,A,B,C,A,A,B,D,A,A,A
4,190001004631,2019,Juruti,PA,23,M,1,3,1,Juruti,...,A,C,B,A,B,B,D,A,A,A


<h2> 4 - Variáveis que são partes das notas da redação </h2>

In [11]:
redacao_parcial = list(df.columns.where(df.columns.str.match('NU_NOTA_COMP')).dropna())
print(f'{len(redacao_parcial)} colunas')

redacao_parcial

5 colunas


['NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5']

<center><font color="red"><b>Para prever a nota da redação, não faz sentido ter as notas parciais</b></font></center>

<center>Pelo mesmo motivo de não usar as respostas dos participantes em cada prova</center>

| Variável | Descrição |
|---|---|
| NU_NOTA_COMP1 | Nota da competência 1 - Demonstrar domínio da modalidade escrita formal da Língua Portuguesa. |
| NU_NOTA_COMP2 | Nota da competência 2 - Compreender a proposta de redação e aplicar conceitos das várias áreas de conhecimento para desenvolver o tema, dentro dos limites estruturais do texto dissertativo-argumentativo em prosa. |
| NU_NOTA_COMP3 | Nota da competência 3 - Selecionar, relacionar, organizar e interpretar informações, fatos, opiniões e argumentos em defesa de um ponto de vista. |
| NU_NOTA_COMP4 | Nota da competência 4 - Demonstrar conhecimento dos mecanismos linguísticos necessários para a construção da argumentação. |
| NU_NOTA_COMP5 | Nota da competência 5 - Elaborar proposta de intervenção para o problema abordado, respeitando os direitos humanos. |

In [12]:
mantem = list(df.columns.where(
    ~df.columns.str.startswith('CO') & ~df.columns.str.startswith('TX') &
    ~df.columns.str.match('Q01[3-9]|Q02[01]') & ~df.columns.str.startswith('NU_NOTA_COMP')
).dropna())

df[mantem].head()

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,...,Q007,Q008,Q009,Q010,Q011,Q012,Q022,Q023,Q024,Q025
0,190001004627,2019,Santarém,PA,21,M,1,3,4,NaN,...,A,B,C,A,A,B,D,A,B,A
1,190001004628,2019,Mãe do Rio,PA,16,F,1,3,1,Paragominas,...,A,B,C,A,A,B,B,A,A,A
2,190001004629,2019,Paragominas,PA,18,F,1,1,1,Paragominas,...,D,E,E,C,A,B,D,A,C,B
3,190001004630,2019,São Sebastião da Boa Vista,PA,23,M,0,3,1,Belém,...,A,B,C,A,A,B,D,A,A,A
4,190001004631,2019,Juruti,PA,23,M,1,3,1,Juruti,...,A,C,B,A,B,B,D,A,A,A


<h2> 5 - <code>NU_INSCRICAO</code> e <code>NU_ANO</code> </h2>

<center>
    Acho que essas colunas podem ser tiradas também.
</center>

| Variável | Descrição |
|---|---|
| NU_INSCRICAO | Número de inscrição  |
| NU_ANO | Ano do Enem |


**OBS:** Caso mais de um ano seja utilizado, aí precisa manter o `NU_ANO`.

In [13]:
mantem = list(df.columns.where(
    ~df.columns.str.startswith('CO') & ~df.columns.str.startswith('TX') &
    ~df.columns.str.match('Q01[3-9]|Q02[01]') & ~df.columns.str.startswith('NU_NOTA_COMP') &
    ~df.columns.str.startswith('NU_INSCRICAO') & ~df.columns.str.startswith('NU_ANO')
).dropna())

df[mantem].head()

,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,...,Q007,Q008,Q009,Q010,Q011,Q012,Q022,Q023,Q024,Q025
0,Santarém,PA,21,M,1,3,4,NaN,NaN,1,...,A,B,C,A,A,B,D,A,B,A
1,Mãe do Rio,PA,16,F,1,3,1,Paragominas,PA,3,...,A,B,C,A,A,B,B,A,A,A
2,Paragominas,PA,18,F,1,1,1,Paragominas,PA,2,...,D,E,E,C,A,B,D,A,C,B
3,São Sebastião da Boa Vista,PA,23,M,0,3,1,Belém,PA,2,...,A,B,C,A,A,B,D,A,A,A
4,Juruti,PA,23,M,1,3,1,Juruti,PA,1,...,A,C,B,A,B,B,D,A,A,A


<h2> 6 - Colunas com muitos valores faltantes (NaN)</h2>

In [14]:
df[mantem].isnull().sum().sort_values(ascending=False).head(20)

TP_SIT_FUNC_ESC            4020
TP_LOCALIZACAO_ESC         4020
SG_UF_ESC                  4020
NO_MUNICIPIO_ESC           4020
TP_DEPENDENCIA_ADM_ESC     4020
TP_ENSINO                  1497
NU_NOTA_CN                 1085
NU_NOTA_MT                 1085
NU_NOTA_LC                  908
NU_NOTA_CH                  908
TP_STATUS_REDACAO           908
NU_NOTA_REDACAO             908
NO_MUNICIPIO_NASCIMENTO     167
SG_UF_NASCIMENTO            167
IN_LACTANTE                   0
IN_GESTANTE                   0
IN_OUTRA_DEF                  0
IN_VISAO_MONOCULAR            0
IN_AUTISMO                    0
IN_DISCALCULIA                0
dtype: int64

<center><font color="red"><b>Os dados relacionados a escola podem ter bastante NaN</b></font></center>
<center>Então, as colunas relacionadas aos dados da escola podem ser removidas</center>

| Variável | Descrição |
|---|---|
| NO_MUNICIPIO_ESC | Nome do município da escola |
| SG_UF_ESC | Sigla da Unidade da Federação da escola |
| TP_DEPENDENCIA_ADM_ESC | Dependência administrativa (Escola) |
| TP_LOCALIZACAO_ESC | Localização (Escola) |
| TP_SIT_FUNC_ESC | Situação de funcionamento (Escola) |

<font color="brown"><b>Atenção:</b></font> O `TP_ENSINO` ainda não dá pra tirar conclusões, dado que a primeira inspeção foi feita sobre 5000 linhas de dados. Devemos olhar a distribuição de NaN conforme o volume de dados aumentar.

In [15]:
mantem = list(df.columns.where(
    ~df.columns.str.startswith('CO') & ~df.columns.str.startswith('TX') &
    ~df.columns.str.match('Q01[3-9]|Q02[01]') & ~df.columns.str.startswith('NU_NOTA_COMP') &
    ~df.columns.str.startswith('NU_INSCRICAO') & ~df.columns.str.startswith('NU_ANO') &
    ~df.columns.str.endswith('_ESC')
).dropna())

df[mantem].head()

,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,...,Q007,Q008,Q009,Q010,Q011,Q012,Q022,Q023,Q024,Q025
0,Santarém,PA,21,M,1,3,4,NaN,NaN,1,...,A,B,C,A,A,B,D,A,B,A
1,Mãe do Rio,PA,16,F,1,3,1,Paragominas,PA,3,...,A,B,C,A,A,B,B,A,A,A
2,Paragominas,PA,18,F,1,1,1,Paragominas,PA,2,...,D,E,E,C,A,B,D,A,C,B
3,São Sebastião da Boa Vista,PA,23,M,0,3,1,Belém,PA,2,...,A,B,C,A,A,B,D,A,A,A
4,Juruti,PA,23,M,1,3,1,Juruti,PA,1,...,A,C,B,A,B,B,D,A,A,A


<h2> 7 - Colunas que indicam o tipo do recurso </h2>

In [16]:
recurso = list(d['recurso'].keys())
print(f'{len(recurso)} colunas')

print(recurso[:3] + ['...'] + recurso[-2:])

recurso.remove('IN_SEM_RECURSO')
series_recurso = df[recurso].agg(lambda x: x.any(), axis='columns').astype('int64')
series_recurso.name = 'IN_RECURSO'

recurso = list(d['recurso'].keys())

34 colunas
['IN_SEM_RECURSO', 'IN_BRAILLE', 'IN_AMPLIADA_24', '...', 'IN_MATERIAL_ESPECIFICO', 'IN_NOME_SOCIAL']


<center><font color="red"><b>Os dados de recurso podem ser juntados em uma coluna só</b></font></center>
<center>Assim, cria-se uma coluna para indicar que foi pedido um dos recursos disponibilizados</center>

| Variável | Descrição |
|---|---|
| IN_BRAILLE | Indicador de solicitação de prova em braille5 |
| IN_AMPLIADA_24 | Indicador de solicitação de prova superampliada com fonte tamanho 245 |
| IN_AMPLIADA_18 | Indicador de solicitação de prova ampliada com fonte tamanho 185 |
| IN_LEDOR | Indicador de solicitação de auxílio para leitura (ledor)5 |
| IN_ACESSO | Indicador de solicitação de sala de fácil acesso5 |
| ... | ... |

<!--
| IN_TRANSCRICAO | Indicador de solicitação de auxílio para transcrição5 |
| IN_LIBRAS | Indicador de solicitação de Tradutor- Intérprete Libras5 |
| IN_TEMPO_ADICIONAL | Indicador de solicitação de tempo adicional5 |
| IN_LEITURA_LABIAL | Indicador de solicitação de leitura labial5 |
| IN_MESA_CADEIRA_RODAS | Indicador de solicitação de mesa para cadeira de rodas5 |
| IN_MESA_CADEIRA_SEPARADA | Indicador de solicitação de mesa e cadeira separada5 |
| IN_APOIO_PERNA | Indicador de solicitação de apoio de perna e pé5 |
| IN_GUIA_INTERPRETE | Indicador de solicitação de guia intérprete5 |
| IN_COMPUTADOR | Indicador de solicitação de computador5 |
| IN_CADEIRA_ESPECIAL | Indicador de solicitação de cadeira especial5 |
| IN_CADEIRA_CANHOTO | Indicador de solicitação de cadeira para canhoto5 |
| IN_CADEIRA_ACOLCHOADA | Indicador de solicitação de cadeira acolchoada5 |
| IN_PROVA_DEITADO | Indicador de solicitação para fazer prova deitado em maca ou mobiliário similar5 |
| IN_MOBILIARIO_OBESO | Indicador de solicitação de mobiliário adequado para obeso5 |
| IN_LAMINA_OVERLAY | Indicador de solicitação de lâmina overlay5 |
| IN_PROTETOR_AURICULAR | Indicador de solicitação de protetor auricular5 |
| IN_MEDIDOR_GLICOSE | Indicador de solicitação de medidor de glicose e/ou aplicação de insulina5 |
| IN_MAQUINA_BRAILE | Indicador de solicitação de máquina Braile e/ou Reglete e Punção5 |
| IN_SOROBAN | Indicador de solicitação de soroban5 |
| IN_MARCA_PASSO | Indicador de solicitação de marca-passo (impeditivo de uso de detector de metais)5 |
| IN_SONDA | Indicador de solicitação de sonda com troca periódica5 |
| IN_MEDICAMENTOS | Indicador de solicitação de medicamentos5 |
| IN_SALA_INDIVIDUAL | Indicador de solicitação de sala especial individual5 |
| IN_SALA_ESPECIAL | Indicador de solicitação de sala especial até 20 participantes5 |
| IN_SALA_ACOMPANHANTE | Indicador de solicitação de sala reservada para acompanhantes5 |
| IN_MOBILIARIO_ESPECIFICO | Indicador de solicitação de mobiliário específico5 |
| IN_MATERIAL_ESPECIFICO | Indicador de solicitação de material específico5 |
| IN_NOME_SOCIAL | Indicador de inscrito que se declarou travesti, transexual ou transgênero e solicitou atendimento pelo Nome Social, conforme é reconhecido socialmente em consonância com sua identidade de gênero6 |
-->

In [17]:
mantem = list(df.columns.where(
    ~df.columns.str.startswith('CO') & ~df.columns.str.startswith('TX') &
    ~df.columns.str.match('Q01[3-9]|Q02[01]') & ~df.columns.str.startswith('NU_NOTA_COMP') &
    ~df.columns.str.startswith('NU_INSCRICAO') & ~df.columns.str.startswith('NU_ANO') &
    ~df.columns.str.endswith('_ESC') & ~df.columns.isin(recurso)
).dropna())

df[mantem].join(series_recurso).head()

,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,...,Q008,Q009,Q010,Q011,Q012,Q022,Q023,Q024,Q025,IN_RECURSO
0,Santarém,PA,21,M,1,3,4,NaN,NaN,1,...,B,C,A,A,B,D,A,B,A,0
1,Mãe do Rio,PA,16,F,1,3,1,Paragominas,PA,3,...,B,C,A,A,B,B,A,A,A,0
2,Paragominas,PA,18,F,1,1,1,Paragominas,PA,2,...,E,E,C,A,B,D,A,C,B,0
3,São Sebastião da Boa Vista,PA,23,M,0,3,1,Belém,PA,2,...,B,C,A,A,B,D,A,A,A,0
4,Juruti,PA,23,M,1,3,1,Juruti,PA,1,...,C,B,A,B,B,D,A,A,A,0


<h2> 8 - Colunas que indicam pedido de atendimento específico </h2>

In [18]:
especifico = list(d['especifico'].keys())
print(f'{len(especifico)} colunas')

display(especifico)

series_especifico = df[especifico].agg(lambda x: x.any(), axis='columns').astype('int64')
series_especifico.name = 'IN_ESPECIFICO'

4 colunas


['IN_GESTANTE', 'IN_LACTANTE', 'IN_IDOSO', 'IN_ESTUDA_CLASSE_HOSPITALAR']

<center><font color="red"><b>Os dados podem ser juntados em uma coluna só</b></font></center>
<center>Assim, cria-se uma coluna para indicar que foi pedido atendimento específico</center>

| Variável | Descrição |
|---|---|
| IN_GESTANTE | Indicador de gestante5 |
| IN_LACTANTE | Indicador de lactante5 |
| IN_IDOSO | Indicador de inscrito idoso5 |
| IN_ESTUDA_CLASSE_HOSPITALAR | Indicador de inscrição em Unidade Hospitalar5 |

In [19]:
mantem = list(df.columns.where(
    ~df.columns.str.startswith('CO') & ~df.columns.str.startswith('TX') &
    ~df.columns.str.match('Q01[3-9]|Q02[01]') & ~df.columns.str.startswith('NU_NOTA_COMP') &
    ~df.columns.str.startswith('NU_INSCRICAO') & ~df.columns.str.startswith('NU_ANO') &
    ~df.columns.str.endswith('_ESC') & ~df.columns.isin(recurso) &
    ~df.columns.isin(especifico)
).dropna())

df[mantem].join(series_recurso).join(series_especifico).head()

,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,...,Q009,Q010,Q011,Q012,Q022,Q023,Q024,Q025,IN_RECURSO,IN_ESPECIFICO
0,Santarém,PA,21,M,1,3,4,NaN,NaN,1,...,C,A,A,B,D,A,B,A,0,0
1,Mãe do Rio,PA,16,F,1,3,1,Paragominas,PA,3,...,C,A,A,B,B,A,A,A,0,0
2,Paragominas,PA,18,F,1,1,1,Paragominas,PA,2,...,E,C,A,B,D,A,C,B,0,0
3,São Sebastião da Boa Vista,PA,23,M,0,3,1,Belém,PA,2,...,C,A,A,B,D,A,A,A,0,0
4,Juruti,PA,23,M,1,3,1,Juruti,PA,1,...,B,A,B,B,D,A,A,A,0,0


<h2> 9 - Colunas que indicam pedido de atendimento especializado (deficiências) </h2>

In [20]:
especializado = list(d['especializado'].keys())
print(f'{len(especializado)} colunas')

display(especializado)

series_especializado = df[especializado].agg(lambda x: x.any(), axis='columns').astype('int64')
series_especializado.name = 'IN_ESPECIALIZADO'

13 colunas


['IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DEFICIENCIA_AUDITIVA',
 'IN_SURDO_CEGUEIRA',
 'IN_DEFICIENCIA_FISICA',
 'IN_DEFICIENCIA_MENTAL',
 'IN_DEFICIT_ATENCAO',
 'IN_DISLEXIA',
 'IN_DISCALCULIA',
 'IN_AUTISMO',
 'IN_VISAO_MONOCULAR',
 'IN_OUTRA_DEF']

<center><font color="red"><b>Os dados podem ser juntados em uma coluna só</b></font></center>
<center>Assim, cria-se uma coluna para indicar que foi pedido atendimento especializado</center>

| Variável | Descrição |
|---|---|
| IN_BAIXA_VISAO | Indicador de baixa visão5 |
| IN_CEGUEIRA | Indicador de cegueira5 |
| IN_SURDEZ | Indicador de surdez5 |
| IN_DEFICIENCIA_AUDITIVA | Indicador de deficiência auditiva5 |
| IN_SURDO_CEGUEIRA | Indicador de surdo-cegueira5 |
| IN_DEFICIENCIA_FISICA | Indicador de deficiência física5 |
| IN_DEFICIENCIA_MENTAL | Indicador de deficiência mental5 |
| IN_DEFICIT_ATENCAO | Indicador de déficit de atenção5 |
| IN_DISLEXIA | Indicador de dislexia5 |
| IN_DISCALCULIA | Indicador de discalculia5 |
| IN_AUTISMO | Indicador de autismo5 |
| IN_VISAO_MONOCULAR | Indicador de visão monocular5 |
| IN_OUTRA_DEF | Indicador de outra deficiência ou condição especial5 |

In [21]:
mantem = list(df.columns.where(
    ~df.columns.str.startswith('CO') & ~df.columns.str.startswith('TX') &
    ~df.columns.str.match('Q01[3-9]|Q02[01]') & ~df.columns.str.startswith('NU_NOTA_COMP') &
    ~df.columns.str.startswith('NU_INSCRICAO') & ~df.columns.str.startswith('NU_ANO') &
    ~df.columns.str.endswith('_ESC') & ~df.columns.isin(recurso) &
    ~df.columns.isin(especifico) & ~df.columns.isin(especializado)
).dropna())

df[mantem].join(series_recurso).join(series_especifico).join(series_especializado).head()

,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,...,Q010,Q011,Q012,Q022,Q023,Q024,Q025,IN_RECURSO,IN_ESPECIFICO,IN_ESPECIALIZADO
0,Santarém,PA,21,M,1,3,4,NaN,NaN,1,...,A,A,B,D,A,B,A,0,0,0
1,Mãe do Rio,PA,16,F,1,3,1,Paragominas,PA,3,...,A,A,B,B,A,A,A,0,0,0
2,Paragominas,PA,18,F,1,1,1,Paragominas,PA,2,...,C,A,B,D,A,C,B,0,0,0
3,São Sebastião da Boa Vista,PA,23,M,0,3,1,Belém,PA,2,...,A,A,B,D,A,A,A,0,0,0
4,Juruti,PA,23,M,1,3,1,Juruti,PA,1,...,A,B,B,D,A,A,A,0,0,0


In [22]:
original_df = df

In [23]:
novas_colunas = pd.DataFrame([series_especializado, series_especifico, series_recurso]).T
df = pd.concat([original_df, novas_colunas], axis=1)

mantem = list(df.columns.where(
    ~df.columns.str.startswith('CO') & ~df.columns.str.startswith('TX') &
    ~df.columns.str.match('Q01[3-9]|Q02[01]') & ~df.columns.str.startswith('NU_NOTA_COMP') &
    ~df.columns.str.startswith('NU_INSCRICAO') & ~df.columns.str.startswith('NU_ANO') &
    ~df.columns.str.endswith('_ESC') & ~df.columns.isin(recurso) &
    ~df.columns.isin(especifico) & ~df.columns.isin(especializado)
).dropna())
df[mantem].head()

,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,...,Q010,Q011,Q012,Q022,Q023,Q024,Q025,IN_ESPECIALIZADO,IN_ESPECIFICO,IN_RECURSO
0,Santarém,PA,21,M,1,3,4,NaN,NaN,1,...,A,A,B,D,A,B,A,0,0,0
1,Mãe do Rio,PA,16,F,1,3,1,Paragominas,PA,3,...,A,A,B,B,A,A,A,0,0,0
2,Paragominas,PA,18,F,1,1,1,Paragominas,PA,2,...,C,A,B,D,A,C,B,0,0,0
3,São Sebastião da Boa Vista,PA,23,M,0,3,1,Belém,PA,2,...,A,A,B,D,A,A,A,0,0,0
4,Juruti,PA,23,M,1,3,1,Juruti,PA,1,...,A,B,B,D,A,A,A,0,0,0


<h2> 10 - Colunas que indicam presença nas provas </h2>

In [24]:
presenca = list(df.columns.where(df.columns.str.startswith('TP_PRESENCA')).dropna())
print(f'{len(presenca)} colunas')

presenca

4 colunas


['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT']

<center><font color="red"><b>Não há necessidade de manter informação sobre a presença</b></font></center>
<center>A nota com valor <code>NaN</code> já indica a falta</center>

| Variável | Descrição |
|---|---|
|TP_PRESENCA_CN|Presença na prova objetiva de Ciências da Natureza|
|TP_PRESENCA_CH|Presença na prova objetiva de Ciências Humanas|
|TP_PRESENCA_LC|Presença na prova objetiva de Linguagens e Códigos|
|TP_PRESENCA_MT|Presença na prova objetiva de Matemática|

In [25]:
mantem = list(df.columns.where(
    ~df.columns.str.startswith('CO') & ~df.columns.str.startswith('TX') &
    ~df.columns.str.match('Q01[3-9]|Q02[01]') & ~df.columns.str.startswith('NU_NOTA_COMP') &
    ~df.columns.str.startswith('NU_INSCRICAO') & ~df.columns.str.startswith('NU_ANO') &
    ~df.columns.str.endswith('_ESC') & ~df.columns.isin(recurso) &
    ~df.columns.isin(especifico) & ~df.columns.isin(especializado) &
    ~df.columns.str.startswith('TP_PRESENCA')
).dropna())
df[mantem].head()

,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,...,Q010,Q011,Q012,Q022,Q023,Q024,Q025,IN_ESPECIALIZADO,IN_ESPECIFICO,IN_RECURSO
0,Santarém,PA,21,M,1,3,4,NaN,NaN,1,...,A,A,B,D,A,B,A,0,0,0
1,Mãe do Rio,PA,16,F,1,3,1,Paragominas,PA,3,...,A,A,B,B,A,A,A,0,0,0
2,Paragominas,PA,18,F,1,1,1,Paragominas,PA,2,...,C,A,B,D,A,C,B,0,0,0
3,São Sebastião da Boa Vista,PA,23,M,0,3,1,Belém,PA,2,...,A,A,B,D,A,A,A,0,0,0
4,Juruti,PA,23,M,1,3,1,Juruti,PA,1,...,A,B,B,D,A,A,A,0,0,0


In [26]:
print(mantem)

['NO_MUNICIPIO_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'NO_MUNICIPIO_NASCIMENTO', 'SG_UF_NASCIMENTO', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO', 'NO_MUNICIPIO_PROVA', 'SG_UF_PROVA', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q022', 'Q023', 'Q024', 'Q025', 'IN_ESPECIALIZADO', 'IN_ESPECIFICO', 'IN_RECURSO']


<h1><font color="purple">Parte 2:</font> Otimizar uso de memória</h1>

Escolher as colunas a serem usadas diminuiu o uso de memória

In [27]:
def compara_memoria(df1, df2):
    from math import ceil
    b1 = df1.memory_usage().sum()
    b2 = df2.memory_usage().sum()
    print('Tipo'.center(15), '|', 'MB'.center(5), '|', 'KB'.center(10), '|', 'Bytes'.center(10))
    print('Original'.center(15), '|', str(ceil(b1 / (1024**2))).center(5), '|', str(ceil(b1 / (1024))).center(10), '|', str(ceil(b1)).center(10))
    print('Limpo'.center(15), '|', str(ceil(b2 / (1024**2))).center(5), '|', str(ceil(b2 / (1024))).center(10), '|', str(ceil(b2)).center(10))
    

In [28]:
df_copy = df.copy()

In [29]:
print('Compara o dataframe original e o com as colunas selecionadas.\n')
compara_memoria(df, df[mantem])

Compara o dataframe original e o com as colunas selecionadas.

      Tipo      |   MB  |     KB     |   Bytes   
    Original    |   6   |    5430    |  5560128  
     Limpo      |   2   |    1641    |  1680128  


════════════════════════════════════════════════════════════════════════════════════

Logo, pode ser interessante trocar algumas coisas do <b><font face="monospace">DataFrame</font></b> para economizar memória e assim conseguir ler mais linhas do csv.

════════════════════════════════════════════════════════════════════════════════════

<h3>I. Trocar o tamanho das variáveis numéricas</h3>

Olhando o arquivo `get_dicionario.ipynb`, o maior tamanho de variável numérica é 12 e de variável alfanumérica é 150. Somente a variável numérica `NU_INSCRICAO` usa 12 dígitos, as demais usam 9 pra baixo. Assim, podemos usar um `int32` e `float32`.

In [30]:
trocar = df.select_dtypes(include=['int64']).columns.to_list()
trocar.remove('NU_INSCRICAO')
df_copy[trocar] = df[trocar].astype('int32')

trocar = df.select_dtypes(include=['float64']).columns.to_list()
df_copy[trocar] = df[trocar].astype('float32')


In [31]:
print('Compara o dataframe original e o com a mudança das variáveis numéricas.\n')
compara_memoria(df, df_copy)

print('---------------------------------------------------------------------------')

print('Verificando com as colunas a ser mantidas.\n')
compara_memoria(df[mantem], df_copy[mantem])

Compara o dataframe original e o com a mudança das variáveis numéricas.

      Tipo      |   MB  |     KB     |   Bytes   
    Original    |   6   |    5430    |  5560128  
     Limpo      |   4   |    3536    |  3620128  
---------------------------------------------------------------------------
Verificando com as colunas a ser mantidas.

      Tipo      |   MB  |     KB     |   Bytes   
    Original    |   2   |    1641    |  1680128  
     Limpo      |   2   |    1251    |  1280128  


<h3>II. Trocar os tipos das categorias </h3>

Com base no arquivo `categoria.json`

In [32]:
import re

dict_categoria = { coluna: pd.CategoricalDtype(
        categories=[ int(item) if item.isdigit() else item for item in list(categoria[coluna]) ],
        ordered=bool(re.match('Q00[5-9]|Q0[12][0-9]', coluna))
    ) for coluna in categoria
}
df_copy = df_copy.astype(dict_categoria)

In [33]:
print('Compara o dataframe original com mudanças das variáveis numéricas + categoria.\n')
compara_memoria(df, df_copy)

print('---------------------------------------------------------------------------')

print('Verificando com as colunas a ser mantidas.\n')
compara_memoria(df[mantem], df_copy[mantem])

Compara o dataframe original com mudanças das variáveis numéricas + categoria.

      Tipo      |   MB  |     KB     |   Bytes   
    Original    |   6   |    5430    |  5560128  
     Limpo      |   2   |    1627    |  1666000  
---------------------------------------------------------------------------
Verificando com as colunas a ser mantidas.

      Tipo      |   MB  |     KB     |   Bytes   
    Original    |   2   |    1641    |  1680128  
     Limpo      |   1   |    550     |   562192  


In [34]:
old_df = df
df = df_copy

<h3>III. Remover linhas</h3>

Remover linhas de participantes que não tenham notam em nenhuma das provas ou redação. Ou seja, todas as notas são `NaN`

In [35]:
limpa_linhas = df[mantem].where(
    (df[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']].notnull()).agg(
        lambda x: x.any(), axis='columns'
    )
).dropna(how='all')

limpa_linhas.head()

,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NO_MUNICIPIO_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,...,Q010,Q011,Q012,Q022,Q023,Q024,Q025,IN_ESPECIALIZADO,IN_ESPECIFICO,IN_RECURSO
0,Santarém,PA,21.0,M,1,3,4,NaN,NaN,1,...,A,A,B,D,A,B,A,0.0,0.0,0.0
1,Mãe do Rio,PA,16.0,F,1,3,1,Paragominas,PA,3,...,A,A,B,B,A,A,A,0.0,0.0,0.0
2,Paragominas,PA,18.0,F,1,1,1,Paragominas,PA,2,...,C,A,B,D,A,C,B,0.0,0.0,0.0
5,Ananindeua,PA,31.0,M,1,1,1,Belém,PA,1,...,A,A,B,D,A,A,A,0.0,0.0,0.0
6,Marabá,PA,30.0,F,2,3,1,Marabá,PA,1,...,A,B,B,B,A,A,B,0.0,0.0,0.0


In [36]:
compara_memoria(df[mantem], limpa_linhas)

      Tipo      |   MB  |     KB     |   Bytes   
    Original    |   1   |    550     |   562192  
     Limpo      |   1   |    547     |   559484  
